In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
import seaborn as sns

In [3]:
df = pd.read_csv('./matches.csv',index_col=0)

df_initial = df
df.head()

In [4]:
# the data set contains only teams which plain premier league.
df['comp'].unique()

array(['Premier League'], dtype=object)

In [5]:
#these are the unique teams present in the dataset.
df['team'].unique()

array(['Manchester City', 'Chelsea', 'Arsenal', 'Tottenham Hotspur',
       'Manchester United', 'West Ham United', 'Wolverhampton Wanderers',
       'Newcastle United', 'Leicester City', 'Brighton and Hove Albion',
       'Brentford', 'Southampton', 'Crystal Palace', 'Aston Villa',
       'Leeds United', 'Burnley', 'Everton', 'Watford', 'Norwich City',
       'Liverpool', 'Fulham', 'West Bromwich Albion', 'Sheffield United'],
      dtype=object)

In [6]:
#the no.of unique teams are 23 as there are 2 seasons data where 3 teams get relegated in each season.
df['team'].unique().size

23

In [7]:
#datatypes of the values present in the dataset , we need to covert every column to a numerical representation.
df.dtypes

date             object
time             object
comp             object
round            object
day              object
venue            object
result           object
gf              float64
ga              float64
opponent         object
xg              float64
xga             float64
poss            float64
attendance      float64
captain          object
formation        object
referee          object
match report     object
notes           float64
sh              float64
sot             float64
dist            float64
fk              float64
pk              float64
pkatt           float64
season            int64
team             object
dtype: object

In [8]:
#converting time to date time type.
df['date'] = df['date'].apply(lambda x: pd.to_datetime(x))
df['date'].dtype

dtype('<M8[ns]')

In [9]:
#creating match kickoff hours and minutes column from time and coverting it into int.
df['kf_hrs'] = df['time'].apply(lambda x: int(x.split(':')[0]))
df['kf_mins'] = df['time'].apply(lambda x: int(x.split(':')[1]))
df['kf_hrs'].dtype , df['kf_mins'].dtype

(dtype('int64'), dtype('int64'))

In [10]:
df['matchweek'] = df['round'].apply(lambda x:int(x.split(" ")[1]))
df['matchweek'].unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 31, 32, 30, 34, 33,
       35, 36, 37, 38], dtype=int64)

In [11]:
df.isnull().sum()

date               0
time               0
comp               0
round              0
day                0
venue              0
result             0
gf                 0
ga                 0
opponent           0
xg                 0
xga                0
poss               0
attendance       696
captain            0
formation          0
referee            0
match report       0
notes           1389
sh                 0
sot                0
dist               1
fk                 0
pk                 0
pkatt              0
season             0
team               0
kf_hrs             0
kf_mins            0
matchweek          0
dtype: int64

In [12]:
#we can remove notes  and match report
df = df.drop(['notes'],axis=1)
df=df.drop(['match report'],axis=1)

In [13]:
#filling NaN value of dist with 0.
df[df['dist'].isna()]
df['dist'] = df['dist'].fillna(0)

In [14]:
df.isnull().sum()

date            0
time            0
comp            0
round           0
day             0
venue           0
result          0
gf              0
ga              0
opponent        0
xg              0
xga             0
poss            0
attendance    696
captain         0
formation       0
referee         0
sh              0
sot             0
dist            0
fk              0
pk              0
pkatt           0
season          0
team            0
kf_hrs          0
kf_mins         0
matchweek       0
dtype: int64

In [15]:
#fill na values with the mean attendance of the team.
attendance_NantemaList = df['team'].unique()
mean_dict = {}
for i in attendance_NantemaList:
    temp_List=[]
    temp_List=  df[df['team']==i]['attendance']
    mean_dict.update({i:temp_List.mean()})

In [16]:
attendance_arr_final = []
for i in attendance_NantemaList:
    attendance_arr_final.extend(df[df['team']==i]['attendance'].fillna(mean_dict[i]))


df['attendance'] = attendance_arr_final

In [17]:
df.isnull().sum()

date          0
time          0
comp          0
round         0
day           0
venue         0
result        0
gf            0
ga            0
opponent      0
xg            0
xga           0
poss          0
attendance    0
captain       0
formation     0
referee       0
sh            0
sot           0
dist          0
fk            0
pk            0
pkatt         0
season        0
team          0
kf_hrs        0
kf_mins       0
matchweek     0
dtype: int64

In [18]:
df.dtypes

date          datetime64[ns]
time                  object
comp                  object
round                 object
day                   object
venue                 object
result                object
gf                   float64
ga                   float64
opponent              object
xg                   float64
xga                  float64
poss                 float64
attendance           float64
captain               object
formation             object
referee               object
sh                   float64
sot                  float64
dist                 float64
fk                   float64
pk                   float64
pkatt                float64
season                 int64
team                  object
kf_hrs                 int64
kf_mins                int64
matchweek              int64
dtype: object

In [37]:
#storing the data of past 3 matchs average.
df_temp = pd.DataFrame()
team_groups = df.groupby('team')
colmn = ['gf','ga','sh','sot','dist','fk','pk','pkatt']
new_colmn = [f"rolling_{i}" for i in colmn]
teamnames= df['team'].unique()
def three_match_avg(group , colmn,new_colmn):
    group = group.sort_values('date')
    threemathcesavg = group[colmn].rolling(3,closed='left').mean()
    group[new_colmn] = threemathcesavg
    group = group.dropna(subset=new_colmn)
    return group

for i in teamnames:
    grouped_each =  team_groups.get_group(i)
    indivi_grp = three_match_avg(grouped_each,colmn,new_colmn)
    df_temp= pd.concat([df_temp,indivi_grp],)

df = df_temp

In [54]:
df['result']

5     W
7     D
9     W
11    D
12    L
     ..
38    L
39    L
40    W
41    L
42    W
Name: result, Length: 1320, dtype: object

In [57]:
#lable encoding datas.
df['day_lable'] = df['day'].astype('category').cat.codes
df['venue_lable'] = df['venue'].astype('category').cat.codes
df['opponent_lable'] = df['opponent'].astype('category').cat.codes
df['formation_lable'] = df['formation'].astype('category').cat.codes
df['referee_lable'] = df['referee'].astype('category').cat.codes
df['result_lable'] = df['result'].apply(lambda x: 1 if x=='W' else 0) 

In [58]:
df.head()

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,rolling_dist,rolling_fk,rolling_pk,rolling_pkatt,day_lable,venue_lable,opponent_lable,formation_lable,referee_lable,result_lable
5,2020-10-17,17:30,Premier League,Matchweek 5,Sat,Home,W,1.0,0.0,Arsenal,...,18.900000,1.333333,0.333333,0.333333,2,1,0,4,3,1
7,2020-10-24,12:30,Premier League,Matchweek 6,Sat,Away,D,1.0,1.0,West Ham,...,17.733333,0.666667,0.000000,0.000000,2,0,21,10,2,0
9,2020-10-31,12:30,Premier League,Matchweek 7,Sat,Away,W,1.0,0.0,Sheffield Utd,...,18.233333,0.666667,0.000000,0.000000,2,0,16,10,14,1
11,2020-11-08,16:30,Premier League,Matchweek 8,Sun,Home,D,1.0,1.0,Liverpool,...,18.466667,1.000000,0.000000,0.000000,3,1,11,7,4,0
12,2020-11-21,17:30,Premier League,Matchweek 9,Sat,Away,L,0.0,2.0,Tottenham,...,19.366667,1.000000,0.000000,0.333333,2,0,18,10,16,0


In [59]:
df1 = df.drop(['time','comp','round','day','venue','captain','formation','season','opponent','referee','team','result','gf','ga','sh', 'sot', 'dist', 'fk', 'pk',
       'pkatt'],axis=1)
df1

,date,xg,xga,poss,attendance,kf_hrs,kf_mins,matchweek,rolling_gf,rolling_ga,...,rolling_dist,rolling_fk,rolling_pk,rolling_pkatt,day_lable,venue_lable,opponent_lable,formation_lable,referee_lable,result_lable
5,2020-10-17,1.5,0.9,59.0,34598.176471,17,30,5,2.000000,2.333333,...,18.900000,1.333333,0.333333,0.333333,2,1,0,4,3,1
7,2020-10-24,1.1,0.5,70.0,34598.176471,12,30,6,1.333333,2.000000,...,17.733333,0.666667,0.000000,0.000000,2,0,21,10,2,0
9,2020-10-31,1.5,0.5,64.0,34598.176471,12,30,7,1.000000,0.666667,...,18.233333,0.666667,0.000000,0.000000,2,0,16,10,14,1
11,2020-11-08,1.6,1.4,54.0,34598.176471,16,30,8,1.000000,0.333333,...,18.466667,1.000000,0.000000,0.000000,3,1,11,7,4,0
12,2020-11-21,1.3,0.7,67.0,34598.176471,17,30,9,1.000000,0.666667,...,19.366667,1.000000,0.000000,0.333333,2,0,18,10,16,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38,2021-05-02,0.5,2.0,34.0,5666.666667,19,15,34,0.333333,1.333333,...,18.466667,0.000000,0.000000,0.000000,3,0,18,0,0,0
39,2021-05-08,0.7,2.1,50.0,5666.666667,15,0,35,0.333333,1.666667,...,18.300000,0.000000,0.000000,0.000000,2,1,6,4,20,0
40,2021-05-16,1.6,1.3,38.0,5666.666667,19,0,36,0.333333,2.000000,...,15.200000,0.333333,0.000000,0.000000,3,0,7,1,10,1
41,2021-05-19,0.8,1.5,50.0,10000.000000,18,0,37,0.333333,2.000000,...,15.266667,0.333333,0.000000,0.000000,6,0,14,1,19,0


In [60]:
X_train = df1[df1['date']<='2021-12-31']
y_train = df1[df1['date']>'2021-12-31']
X_test = df1[df1['date']<='2021-12-31']['result_lable']
y_test = df1[df1['date']>'2021-12-31']['result_lable']
X_train=X_train.drop(['date','result_lable'],axis=1)
y_train=y_train.drop(['date','result_lable'],axis=1)
print(len(X_train),len(X_test),len(y_train),len(y_test))

1038 1038 282 282


In [61]:
model = RandomForestClassifier(n_estimators=550,min_samples_leaf=5,max_leaf_nodes=45,max_depth=70,random_state=1)

In [62]:
model.fit(X_train,X_test)

RandomForestClassifier(max_depth=70, max_leaf_nodes=45, min_samples_leaf=5,
                       n_estimators=550, random_state=1)

In [63]:
pred = model.predict(y_train)
pred

array([1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0,
       1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0,
       0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0,
       0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0,
       0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1,
       1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int64)

In [67]:
acc = accuracy_score(y_test,pred)
acc

0.7198581560283688

In [68]:
precs = precision_score(y_test,pred,average='weighted')
precs

0.7149924064817681

In [69]:
df['result_lable'].unique(),df_initial['result'].unique()

(array([1, 0], dtype=int64), array(['L', 'W', 'D'], dtype=object))

In [70]:
print(X_train.columns)

Index(['xg', 'xga', 'poss', 'attendance', 'kf_hrs', 'kf_mins', 'matchweek',
       'rolling_gf', 'rolling_ga', 'rolling_sh', 'rolling_sot', 'rolling_dist',
       'rolling_fk', 'rolling_pk', 'rolling_pkatt', 'day_lable', 'venue_lable',
       'opponent_lable', 'formation_lable', 'referee_lable'],
      dtype='object')
